In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import gensim
import numpy as np
import pandas as pd
import nltk
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split 
from matplotlib import pyplot as plt 
import seaborn as sn 

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
def build_model():
    model = Word2Vec(corpus_file='tok_subtitles_data', vector_size=100, window=5, min_count=3, workers=4)
    print("Model built.")
    model.save("word2vec.model")

In [3]:
model = Word2Vec.load("word2vec.model")

In [4]:
decades = ["1930s", "1940s", "1950s", "1960s", "1970s", "1980s", "1990s", "2000s", "2010s"]

In [3]:
allmovies = []
current = []
movie_indices = []
  
count = 0
with open('tok_movie_data', 'r') as tok_movies, open('subtitles_data', 'r') as subs:
    subs.readline()
    while True:
        sub_line = subs.readline()
        line = tok_movies.readline()
        if not (line or sub_line):
            allmovies.append(current)
            break
        match = re.search(r"^\*\*\*[0-9]+\,[0-9]+\,[0-9]+\*\*\*$", sub_line)
        if match:
            allmovies.append(current)
            current = []
            count += 1
            movie_indices.append(count)
            sub_line = subs.readline()
            continue
        current.append(line)
        count += 1
        if (count%100000) == 0:
            print(count)


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

In [6]:
# print(len(allmovies))
# print(allmovies[1][0])

In [7]:
# import pandas as pd

# movie_ids = []
# movie_decades = []
# movie_titles = []
# movie_genres = []
# movie_years = []

# metadata = pd.read_excel('movie_metadata.xlsx', sheet_name='Sheet1')
# df = pd.DataFrame(metadata, columns=['textID','title','year','decade','genre'])

# movie_ids = df['textID'].tolist()
# movie_decades = df['decade'].tolist()
# movie_titles = df['title'].tolist()
# movie_genres = df['genre'].tolist()
# movie_years = df['year'].tolist()

In [8]:
# import random

# sample_movie_indices = []
# sample_metadata = []

# for decade in decades:
#     for i in range(450):
#         index = random.randint(0, (len(movie_decades)-1))
#         while (movie_decades[index] != decade) | (index in sample_movie_indices):
#             index = random.randint(0, (len(movie_decades)-1))
#         sample_movie_indices.append(index)
        

In [4]:
import pandas as pd

sample_movie_indices = []
sample_metadata = []

sample_file = pd.read_excel("sample_movie_metadata.xlsx", sheet_name='Sample')
sample_df = pd.DataFrame(sample_file, columns=['index','textID','title','year','decade','genre'])

sample_movie_indices = sample_df['index'].tolist()
sample_movie_indices = [int(i) for i in sample_movie_indices]

ids = sample_df['textID'].tolist()
titles = sample_df['title'].tolist()
decades = sample_df['decade'].tolist()
years = sample_df['year'].tolist()
genres = sample_df['genre'].tolist()

for i in range(len(sample_movie_indices)):
    index = sample_movie_indices[i]
    id = ids[i]
    title = titles[i]
    decade = decades[i]
    year = years[i]
    genre = genres[i]
    meta = [index, id, title, year, decade, genre]
    sample_metadata.append(meta)

In [5]:
print(len(sample_movie_indices))
print(sample_metadata[0])

4050
[168, 25161, 'Gambling Lady', 1934, '1930s', 'Drama']


In [11]:
# movievectors = []

# for i in range(len(sample_movie_indices)):
#     index = sample_movie_indices[i]
#     movie = allmovies[index]
#     totvec = np.zeros(100)
#     for line in movie:
#         for token in line.split(" "):
#             try:
#                 totvec = totvec + model.wv[token]
#             except KeyError:
#                 continue
#     if (i%100 == 0):
#         print(str(i))

#     movievectors.append(totvec)

#     id = movie_ids[index]
#     title = movie_titles[index]
#     decade = movie_decades[index]
#     year = movie_years[index]
#     genre = movie_genres[index]
#     meta = [index, id, title, year, decade, genre]
#     sample_metadata.append(meta)
    

In [12]:
# print(len(movievectors))
# print(movievectors[0])

In [13]:
def write_sample_meta():
    sample_df = pd.DataFrame(sample_metadata, columns=['index','textID','title','year','decade','genre'])
    sample_df.to_excel("sample_movie_metadata.xlsx", sheet_name='Sample', index=False)

In [14]:
def write_movie_vectors():
    with open('movie_vectors', 'w') as f:
        for vector in movievectors:
            f.write(str(vector) + '\n')

In [15]:
movievectors = []
with open('movie_vectors', 'r') as fread:
    file = fread.read().rstrip()
    vectors = file.replace("[", "").strip().split(']')
    for i in range(4050):
        current = np.zeros(100)
        values = vectors[i].split()
        for j in range(100):
            current[j] = current[j] + float(values[j].strip('\n'))
        movievectors.append(current)

In [16]:
print(movievectors[0])

[ -7758.42975128   1437.61948495   6363.02029613   -353.85453451
   2526.86870275  -8162.21084991    804.71707907    175.11357649
 -15940.49886915  -2555.8843161   -6439.0239977   -9985.90603238
  -6357.20584343   4104.28803649  10729.73373431  -7989.6275671
  -5581.82473803   1534.17248715  -6567.51363221  -7116.25713211
   7299.46573964  -1040.08630491  -6500.79773156   1147.78441579
     73.42421195  -2476.93878697   2882.9242971    1643.78764048
 -16911.81955129 -13905.92811035  10349.69969014   2368.63109985
  -2945.86452826 -10079.32786384  -1706.96660779  -7064.21037742
  -7724.46642654  -8832.71270214   -509.67079343 -13050.17396462
  -7090.40294494    404.70277117  -4650.73009597  -2812.97976232
   5304.01430519   1069.29514702  -4753.13754769  -8184.67936857
  -5883.52933817   -640.19291059  -1490.83648687  -3770.9354938
   9709.18187279   6822.22604097    965.81159874  -7461.56350547
   9915.12953046  -6181.30518893   2706.79287654   6309.48009577
   2422.38295088  -4956.004

In [17]:
from sklearn.cluster import KMeans

num_clusters = 30
kmmovies = KMeans(n_clusters=num_clusters, random_state=0)
movieclusters = kmmovies.fit_predict(movievectors)

In [18]:
for i in range(len(movieclusters)):
    if movieclusters[i] == 29:
        print(sample_metadata[i])

[376, 30418, 'Marie Antoinette', 1938, '1930s', 'Biography, Drama, History']
[425, 31725, 'Ninotchka', 1939, '1930s', 'Comedy, Romance']
[444, 32080, 'Union Pacific', 1939, '1930s', 'Drama, Western']
[423, 31679, 'Mr. Smith Goes to Washington', 1939, '1930s', 'Comedy, Drama']
[479, 32484, 'Foreign Correspondent', 1940, '1940s', 'Action, Romance, Thriller']
[971, 40242, 'Command Decision', 1948, '1940s', 'Action, Drama, War']
[795, 37614, 'The Corn Is Green', 1945, '1940s', 'Drama']
[688, 35966, 'Hangmen Also Die!', 1943, '1940s', 'Drama, Film-Noir, Thriller']
[582, 34272, 'That Hamilton Woman', 1941, '1940s', 'Drama, History, Romance, War']
[698, 36172, 'The More the Merrier', 1943, '1940s', 'Comedy, Romance, War']
[1251, 44331, 'Affair in Trinidad', 1952, '1950s', 'Crime, Drama, Film-Noir']
[1826, 53291, 'Some Like It Hot', 1959, '1950s', 'Comedy, Romance']
[1556, 48947, 'Anastasia', 1956, '1950s', 'Biography, Drama, History']
[1150, 42648, 'Kiss Tomorrow Goodbye', 1950, '1950s', 'Cri

In [19]:
def write_clusters():
    with open(('clusters_' + str(num_clusters)), 'w') as f:
        for i in range(num_clusters):
            f.write("\nCLUSTER {num}\n".format(num=i))
            for j in range(len(movieclusters)):
                if movieclusters[j] == i:
                    f.write(str(sample_metadata[j]) + "\n")

In [20]:
# movieclusterdistances = kmmovies.transform(movievectors)

In [21]:
# distances = np.zeros(num_clusters)
# cluster_movies = [None]*num_clusters

# for i in range(len(movieclusterdistances)):
#     current = movieclusterdistances[i]
#     for j in range(len(current)):
#         if distances[j]:
#             if current[j] < distances[j]:
#                 distances[j] = current[j]
#                 cluster_movies[j] = sample_metadata[i]
#         else:
#             distances[j] = current[j]
#             cluster_movies[j] = sample_metadata[i]

In [22]:
# print(distances)
# print(cluster_movies)

In [6]:
with open('sample_subtitles_data', 'w') as f:
    for i in range(len(sample_movie_indices)):
        index = sample_movie_indices[i]
        movie = allmovies[index]
        for line in movie:
#             totvec = np.zeros(100)
            f.write(line.strip('\n').replace(",", "").strip() + ',' + decades[i] + ',' + str(i) + '\n')
#             for token in line.split(" "):
#                 try:
#                     totvec = totvec + model.wv[token]
#                 except KeyError:
#                     continue
#             linevectors.append(totvec)
#         if (i%100 == 0):
#             print(str(i))


In [5]:
# ON COLAB, THESE SHOULD WORK TOTALLY FINE

# Load BERT and the preprocessing model from TF Hub.
bert_preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1')
bert_encoder = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3')

#CREATING BERT MODELS 
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

#SPLIT DATA INTO TRAINING AND TEST SETS
#data = whereever the subtitles are being read from 
dataframe = pd.DataFrame(data, columns=[year for year in decades])
X_train, X_test, Y_train, Y_test = train_test_split()


#CONSTRUCT MODEL 
model = tf.keras.Model(inputs=[text_input], outputs=[outputs])
#pretty sure we now need to pass in a classification task - talk with Grace

METRICS = [ 
    tf.keras.metrics.CategoricalAccuracy(name='Accuracy')
    tf.keras.metrics.Precision(name='Precision')
    tf.keras.metrics.Recall(name='Recall')
]

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=METRICS)
model.fit(X_train, y_train)



NameError: name 'hub' is not defined

In [ ]:
from sklearn import confusion_matrix, classification report 
# visualization for accuracy: This will probably have to be done by decade so not sure if itll work 
cm = confusion_matrix(Y_test, Y_predicted) 
sn.heatmap(cm, annot='true', fmt='d')
plt.xlabel("Predicted")
plt.ylabel("Truth")